In [61]:
import os
from transformers import pipeline
import json
import numpy as np
import pandas as pd
VIDEOS = ['jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp',
 'JFBB_Rhythm_Section_Day1_Morning Thomas',
 'JFBB_Rhythm_Section_Day1_Afternoon Thomas',
 'JFBB_Jazz_Funk_morning_video20230419101818 Luis Cal García',
 'JFBB_Jazz_Funk_afternoon_video20230419143359 Luis Cal García',
 'jazzaar-20230417-afternoon-JFLB-Cuban-Vocals-video Philipp',
 'jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp']

In [95]:
columns = ['segment_id', 'start', 'end', 'text']
dimensions = (-1,4)
for video in VIDEOS:
    output = []
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()

            
            meta_info = [file.split('_')[i] for i in range(3)]
            output = output + meta_info + [text]
            
    output = np.reshape(output, dimensions)
    output_df = pd.DataFrame(output, columns=columns)

    
    output_df=output_df.astype({'start': 'int32', 'end': 'int32', 'text': 'string'})
    output_df=output_df.sort_values(by='start', axis=0)

    output_df['end_l1'] = output_df['end'].shift(1)
    output_df['diff_to_last_snippet'] = output_df['start'] - output_df['end_l1']
    curr_row = output_df.iloc[0]
    rows =[]
    for index, row in output_df.iterrows():
        if row.segment_id == '1':
            continue
        elif row.diff_to_last_snippet < 20:
            curr_row.end = row.end
            curr_row.text = curr_row.text + row.text
        else:
            
            if not os.path.exists(f"../data/transcriptions/processed_merged/{video}"):
                        # Create a new directory because it does not exist
                os.makedirs(f"../data/transcriptions/processed_merged/{video}")

            with open(f'../data/transcriptions//processed_merged/{video}/{curr_row.segment_id}_{curr_row.start}_{curr_row.end}_seg.txt', 'w') as output:
                output.write(curr_row.text)
            rows.append(curr_row)
            curr_row = row


In [96]:

output_df=output_df.astype({'start': 'int32', 'end': 'int32', 'text': 'string'})
output_df=output_df.sort_values(by='start', axis=0)

output_df['end_l1'] = output_df['end'].shift(1)
output_df['diff_to_last_snippet'] = output_df['start'] - output_df['end_l1']
print(len(output_df.loc[output_df['diff_to_last_snippet'] < 20]))
curr_row = output_df.iloc[0]
rows =[]
for index, row in output_df.iterrows():
    if row.segment_id == '1':
        continue
    elif row.diff_to_last_snippet < 20:
        curr_row.end = row.end
        curr_row.text = curr_row.text + row.text
    else:
        rows.append(curr_row)
        curr_row = row


pd.DataFrame(rows).head(15)

21


,segment_id,start,end,text,end_l1,diff_to_last_snippet
53,1,129,153,"So not... We don't do that. Okay, then you sin...",NaN,NaN
63,3,259,284,That's the approach. But I haven't sung anythi...,153.0,106.0
65,5,312,325,"So that might actually be the web, yeah? Elvir...",284.0,28.0
7,6,411,440,"I'm sorry, I don't know what I'm doing. I'm ju...",325.0,86.0
35,8,620,631,"I don't know. I mean, but there's And they kee...",440.0,180.0
43,10,1371,1385,"Can you hear it? I think it's a bit before, bu...",631.0,740.0
19,12,1453,1470,"But easy, you don't have to come in when I'm d...",1385.0,68.0
10,15,1838,1856,"No, I think it's the first one. The first one ...",1470.0,368.0
28,18,2243,2284,I think that was it.And I'm going to explain t...,1856.0,387.0
4,21,2585,2596,"Sooooo, enjoy!",2284.0,301.0
